In [1]:
import os
import tifffile as tf
import numpy as np
import geopandas as gpd
from shapely.geometry import mapping
from rasterio.features import rasterize
from skimage.transform import resize as img_resize       
import cv2
from glob import glob


In [2]:
file_num = 0

# Define original image paths
data_dict = {
        "output-XETG00522__0066398__Region_1__20250515__183305":"Xenium H&E Meso1-ICON2 TMA 5-21-2025_matching_orientation.ome.tif" ,
        "output-XETG00522__0066402__Region_1__20250515__183305":"Xenium H&E PCF TMA 5-28-2025_matching_orientation.ome.tif"
        }

xenium_folder = list(data_dict.keys())[file_num]
slide_name = data_dict[xenium_folder]

# Define annotation paths
root_dir = "/rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration"
he_annot = os.path.join(root_dir, "tma_annotations", slide_name.replace(".ome.tif", "_annot.geojson"))

core_img_dir = os.path.join(root_dir, "palom_registration", slide_name.split(".")[0], "registered_cores").replace(" ", "_")
output_path = os.path.join(root_dir, "palom_registration", slide_name.replace(".ome.tif", "_registered.ome.tif"))

In [3]:
## Helper functions 

def img_resize(img, scale_factor):
    width  = int(np.floor(img.shape[1] * scale_factor))
    height = int(np.floor(img.shape[0] * scale_factor))
    return cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)

def write_ome_tif(filename, image, channel_names,
                  photometric_interp, metadata, subresolutions):
    
    with tf.TiffWriter(filename, bigtiff=True) as tif:
        px_size_x = metadata['PhysicalSizeX']
        px_size_y = metadata['PhysicalSizeY']

        options = dict(
            photometric=photometric_interp,
            tile=(1024, 1024),
            maxworkers=8,
            compression='jpeg',
            compressionargs={'level':85},
            resolutionunit='CENTIMETER',
        )

        print("Writing pyramid level 0")
        tif.write(
            image,
            subifds=subresolutions,
            resolution=(1e4 / px_size_x, 1e4 / px_size_y),
            metadata=metadata,
            **options,
        )

        scale = 1
        for i in range(subresolutions):
            scale *= 0.5
            # down‑sample by 2×
            if photometric_interp == 'minisblack':
                if image.shape[0] < image.shape[-1]:
                    image = np.moveaxis(image,0,-1)
                    image = img_resize(image,0.5)
                    image = np.moveaxis(image,-1,0)
            else:
                image = img_resize(image,0.5)

            print("Writing pyramid level {}".format(i+1))
            tif.write(
                image,
                subfiletype=1,
                resolution=(1e4 / scale / px_size_x, 1e4 / scale / px_size_y),
                **options
            )
            
    print("Saved:", filename)


In [4]:
subres_levels  = 7                                        
core_img_list = sorted(glob(os.path.join(core_img_dir, "*.ome.tif")))
print(f"Found {len(core_img_list)} core images in directory") 
sample_core = core_img_list[0]

Found 28 core images in directory


In [5]:
print(f"Initializing empty slide with {os.path.basename(sample_core)}...") 

with tf.TiffFile(sample_core) as tif:
    reconstructed_slide = tif.asarray()                              # (H, W, 3)

    if not tif.is_ome:
        raise RuntimeError("Input is not OME‑TIFF.")

    meta_dict = tf.xml2dict(tif.ome_metadata)
    px_x   = meta_dict['OME']['Image']['Pixels']['PhysicalSizeX']
    px_y   = meta_dict['OME']['Image']['Pixels']['PhysicalSizeY']
    unit   = meta_dict['OME']['Image']['Pixels']['PhysicalSizeXUnit']
    try:
        channel_names = [
            ch['Name'] for ch in meta_dict['OME']['Image']['Pixels']['Channel']
        ]
    except (KeyError, TypeError) as e:
        channel_names = None

H, W, C = reconstructed_slide.shape
photometric = 'rgb' 
metadata = {
    'PhysicalSizeX': px_x,
    'PhysicalSizeXUnit': unit,
    'PhysicalSizeY': px_y,
    'PhysicalSizeYUnit': unit,
    'Channel': {'Name': channel_names},
}
print("Slide initialized:", reconstructed_slide.shape)

Initializing empty slide with A-4_registered.ome.tif...
Slide initialized: (108707, 48292, 3)


In [8]:
# load core polygons ---------------------------------------
gdf = gpd.read_file(he_annot)
# gdf = gdf[gdf['isMissing'] == False].reset_index(drop=True)

In [9]:
# iterate over cores ---------------------------------------
print("Repopulating cores...")
for idx, row in gdf.iterrows():
    core_id = row.get('name', f'core_{idx + 1}')

    core_path = os.path.join(core_img_dir, f"{core_id}_registered.ome.tif")
    
    assert os.path.exists(core_path), f"Registered core image not found for {core_id}"
    if core_path == sample_core:
        continue
        
    print(f"Adding {core_id} to image reconstruction") 

    with tf.TiffFile(core_path) as tif:
        core_image = tif.asarray()                              # (H, W, 3)

    # Find foreground (background is white space)
    mask_bool = np.any(core_image != [255, 255, 255], axis=-1).astype(bool)

    reconstructed_slide[mask_bool, :] = core_image[mask_bool,:]
    
print(f"\n── Writing reconstructed slide {os.path.basename(output_path)}")
write_ome_tif(
    filename=output_path,
    image=reconstructed_slide,
    channel_names=channel_names,
    photometric_interp=photometric,
    metadata=metadata,
    subresolutions=subres_levels,
)


Repopulating cores...
Adding A-5 to image reconstruction
Adding A-6 to image reconstruction
Adding B-1 to image reconstruction
Adding B-4 to image reconstruction
Adding B-5 to image reconstruction
Adding B-6 to image reconstruction
Adding C-3 to image reconstruction
Adding C-4 to image reconstruction
Adding C-5 to image reconstruction
Adding C-6 to image reconstruction
Adding D-2 to image reconstruction
Adding D-4 to image reconstruction
Adding D-5 to image reconstruction
Adding E-1 to image reconstruction
Adding E-2 to image reconstruction
Adding E-5 to image reconstruction
Adding E-6 to image reconstruction
Adding F-1 to image reconstruction
Adding F-6 to image reconstruction
Adding G-1 to image reconstruction
Adding G-6 to image reconstruction
Adding H-1 to image reconstruction
Adding H-6 to image reconstruction
Adding I-1 to image reconstruction
Adding I-6 to image reconstruction
Adding J-1 to image reconstruction
Adding D-3 to image reconstruction

── Writing reconstructed slide X

In [7]:
sample_core

'/rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/TMP-IL-Pilot/20250515__183240__CIMAC_Validation/registration/palom_registration/Xenium_H&E_Meso1-ICON2_TMA_5-21-2025_matching_orientation/registered_cores/A-4_registered.ome.tif'